In [50]:

import pandas as pd
import torch

In [51]:

# Example DataFrame with columns X, Y, Z, Atom_Name, Residue_Name, Residue_ID, Atom_Type, and Timeframe
# Load nodes for all timesteps
df = pd.read_csv("DATA/FILT_HB/edges2.csv")


In [52]:
max_src_val = df['dst'].max()
print("Maximum src value:", max_src_val)

Maximum src value: 115


In [53]:


# Load the .pt file
data = torch.load("DATA/FILT_HB/node_features.pt")

data.shape

torch.Size([1, 116])